# Philo
-------------------------------------------------------------
A small-scale cold gas VTVL rocket

In [110]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

The goal of this notebook is to outline the process for determining the optimal parameters for a cold gas VTVL rocket.

### Assumptions
1. Isentropic Process
2. Ma = 1 at the throat of the nozzle (proof: [link here])}
3. Pb = 101.325kPa (Ambient Pressure)
4. Tb = 298.15K (Ambient Temperature)

### Initial Vehicle Model
1. Engine throat is the exit nozzle
2. Propellant: Air, cp = 0.846, cv = 0.657, gamma = 1.28
3. Dry mass        = 1.89kg
4. Propellant mass = 0.53kg

In [111]:
# Environment Properties
P_b = 101325             # Pa (Ambient Pressure)
T_b = 298.15             # K (Ambient Temperature)
R = 8.314                # J/(K*mol)
g = 9.81                 # m/s^2

# Propellant Properties 
cp = 1.005               # Constant pressure specific heat
cv = 0.718               # Constant volume specific heat
molar_mass = 0.02897     # kg/mol
gamma = cp/cv            # Specific Heat Ratio

# Vehicle Properties
dry_mass = 1.89                     # kg (rough estimation based on current design)
prop_mass = 0.53                    # kg (based on tank capacity and gas)
wet_mass = dry_mass + prop_mass     # kg

#### Step 0: Determine Theoretical Maximum Flight Time

This will be driven by our initial assumptions and initial vehicle model. With these parameters we can determine the theoretical maximum flight time using the rocket equation and by dividing the delta_V by gravity vector:

\begin{equation}
V_\Delta = V_e * ln\left(\frac{m_o}{m_f} \right)
\end{equation}

TODO: Provide source for V_e = 1000m/s 

In [112]:
v_e_ideal = 1000   # m/s

delta_v = v_e_ideal * np.log(wet_mass/dry_mass)

print ("Delta V: %0.2f m/s" % delta_v)

Delta V: 247.19 m/s


Divide by gravity vector to obtain flight time:

In [113]:
t_theoretical = delta_v / g

print ("Theoretical Flight Time: %0.2f sec" % t_theoretical)

Theoretical Flight Time: 25.20 sec



#### Step 1: Determine Choked Flow Parameters
Determine choked flow parameters:
> <b>KEY CONCEPTS: </b>
> * At choked flow, mass flow rate is <b>constant</b>
> * Since choked flow occurs at the throat of the nozzle, we base our equations relative to this plane. 
> * Choked flow occurs when the chamber pressure and temperature (stagnation pressure and temperature) reach their critical values.
> * To determine these critical values, we must use the critical ratios (a property of the propellant).
> * Thus choked flow is achieved when the chamber pressure (P_o) is greater than or equal to the critical pressure (P_*), and the chamber temperature (T_o) is greater than or equal to the critical temperature (T_*) respectively.

 Find the minimum chamber pressure (P_o) required to achieve choked flow and add margin 
 > Margin is added to compensate for regulator lag and to keep us safely in the choked flow region. If we break out of choked flow, our constant mass flow assumption no longer holds true, making the math much more complicated.

\begin{equation}
\frac{P_*}{P_o} = \frac{2}{\gamma+1}^\frac{\gamma}{\gamma-1}
\end{equation}

\begin{equation}
\frac{T_*}{T_o} = \frac{2}{\gamma+1}
\end{equation}

Since we know the flow is choked when the critical pressure (P_*) equals the back pressure (P_b), we can determine the minimum chamber pressure:

\begin{equation}
\frac{P_b}{P_o} = \frac{2}{\gamma+1}^\frac{\gamma}{\gamma-1}
\end{equation}

\begin{equation}
\frac{T_b}{T_o} = \frac{2}{\gamma+1}
\end{equation}

In [114]:
# Chamber Properties
P_o = P_b/((2/(gamma+1))**(gamma/(gamma-1)))
T_o = T_b/(2/(gamma+1))

print ("Choked Flow:")
print ("Minimum Chamber Pressure: %.2f Pa" % P_o)
print ("Minimum Chamber Temperature: %.2f K" % T_o)

Choked Flow:
Minimum Chamber Pressure: 191784.01 Pa
Minimum Chamber Temperature: 357.74 K


TODO: Ara - reword and clean up below

#### Step 2: Determine Exit Velocity at Nozzle

Now that we know the the minimum chamber pressure and temperature, we can calculate the expected minimum exit velocity at the nozzle (V_e_min) using the following equation:

\begin{equation}
V_e = \sqrt{\frac{T_b R}{M}*\frac{2 \gamma}{\gamma-1}*\left(1-\left(\frac{P_b}{P_o}\right)^{\frac{\gamma-1}{\gamma}}\right)}
\end{equation}

In [115]:
V_e_min = np.sqrt(((T_b * R) / molar_mass) \
      * ((2 * gamma) / (gamma - 1)) \
      * (1 - (P_b / P_o) ** (
        (gamma - 1) / gamma)))

Isp_min = V_e_min / g

print ("Minimum Exit Velocity at Nozzle: %.2f m/s" % V_e_min)
print ("Minimum Specific Impulse: %2f sec" % Isp_min)

Minimum Exit Velocity at Nozzle: 315.94 m/s
Minimum Specific Impulse: 32.205834 sec


#### Step 3: Determine Cross-Sectional Area at the Throat of the Nozzle

Next we need to determine the cross-sectional area of the throat. However, before we proceed, we must first select our desired nominal thrust then solve for the mass flow rate using the following equation:

\begin{equation}
F = \dot{m}*V_e
\end{equation}

Rewritten as:

\begin{equation}
\dot{m} = \frac{F}{V_e}
\end{equation}

In order to determine the desired nominal thrust, we must calcuate the force necessary to null out the acceleration of a fully loaded rocket:

\begin{equation}
F_{nominal} = m_{wet} * g
\end{equation}

Of course, this force will change as the rocket loses mass, but it gives us a place to start.

In [116]:
F_nom = wet_mass * g

print ("Nominal Engine Thrust: %.2f N" % F_nom)

Nominal Engine Thrust: 23.74 N


With a nominal thrust, we can now work out a rough cross-sectional area at the throat.

In [117]:
mass_flow = F_nom / V_e_min         # kg/s

print ("Mass Flow Rate: %.6f kg/s" % mass_flow)

Mass Flow Rate: 0.075142 kg/s


With a known mass flow rate, we can now determine the cross-sectional area at the throat using the following equation:
    
\begin{equation}
A = \frac{\dot{m}* \left(1+(\gamma-1)\frac{M_a^2}{2}\right)^\frac{\gamma+1}{2(\gamma-1)}}{M_a P_o\sqrt{\frac{\gamma}{R T_o}}}
\end{equation}

Since we are solving for the area at the throat of the nozzle: \begin{equation} M_a = 1 \end{equation}

In [118]:
Ma = 1                             # Mach Number
A_t = (mass_flow*(1+(gamma-1)*((Ma**2)/2))**((gamma+1)/(2*(gamma-1))))/(Ma*P_o*np.sqrt(gamma/(R*T_o)))

print ("Cross-sectional Area at Throat: %.8f m^2" % A_t)
print ("Cross-sectional Area at Throat: %.2f mm^2" % (A_t*1e6))

print ("Throat Diameter: %.2f mm" % (2*(np.sqrt((A_t*1e6)/np.pi))) )

Cross-sectional Area at Throat: 0.00003121 m^2
Cross-sectional Area at Throat: 31.21 mm^2
Throat Diameter: 6.30 mm


#### Step 4: Determine Flight Time

We can now obtain a ballpark estimate of flight time by dividing the total propellant mass by mass flow rate

\begin{equation}
t_{flight} = \frac{m_{prop}}{\dot{m}}
\end{equation}

In [119]:
t_flight = prop_mass / mass_flow    # sec

print ("Time of Flight: %0.2f sec" % t_flight)

Time of Flight: 7.05 sec


#### Step 5: Optimize for Flight Time

Not very impressive, but now we have a starting point and there's plenty of room to optimize...

Now that we know the minimum values for achieving choking flow while producing the necessary force to null out vertical acceleration of the vehicle. We can now figure out which knobs to tune to optimize for flight time while checking/validating that we are still at choked flow (i.e. - we need to setup our equations such that mass flow rate reachs a maximum and isn't greater than the theoretical limit [TODO: Ara])
<br><br>
Flight time optimization goals:
1. Increase P_o to increase V_e and decrease mass flow rate and A thus increasing flight time.
3. Optimization shall check choke flow model for compliance
<br><br>

#### Step 6: Further Optimize by Adding an Exit Nozzle

[TODO: Up until this point we've been assuming the throat is the exit nozzle, next step is to add an optimized exit nozzle to maximize V_e]